In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0,1,2,3'

In [ ]:
%run -p ../train_cnf_drop_cifar.py --data cifar10 --dims 64,64,64 --strides 1,1,1,1 --num_blocks 2 --layer_type concat --multiscale True --rademacher True --batch_size 900 --test_batch_size 500 --save ../experiments_published/cnf_conditional_cifar10_bs900_drop_0_5_run1 --resume ../experiments_published/cnf_conditional_cifar10_bs900_drop_0_5_run1/current_checkpt.pth --seed 1 --lr 0.0001 --conditional True --controlled_tol False --train_mode semisup --log_freq 10 --weight_y 0.5 --dropout_rate 0.5
#

/tancode/repos/tan-ffjord/train_cnf_drop_cifar.py
import argparse
import os
import time
import numpy as np

import torch
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as tforms
from torchvision.utils import save_image

import lib.layers as layers
import lib.utils as utils
import lib.multiscale_parallel as multiscale_parallel
import lib.modules as modules
import lib.thops as thops

from train_misc import standard_normal_logprob
from train_misc import set_cnf_options, count_nfe, count_parameters, count_total_time
from train_misc import add_spectral_norm, spectral_norm_power_iteration
from train_misc import create_regularization_fns, get_regularization, append_regularization_to_log

from tensorboardX import SummaryWriter

# go fast boi!!
torch.backends.cudnn.benchmark = True
SOLVERS = ["dopri5", "bdf", "rk4", "midpoint", 'adams', 'explicit_adams']
parser = argparse.ArgumentParser("Continuous Normalizing Flow")
parser.add_argument("--data", c

Files already downloaded and verified
Files already downloaded and verified


ODENVP(
  (transforms): ModuleList(
    (0): StackedCNFLayers(
      (chain): ModuleList(
        (0): LogitTransform()
        (1): CNF(
          (odefunc): RegularizedODEfunc(
            (odefunc): ODEfunc(
              (diffeq): ODEnet(
                (layers): ModuleList(
                  (0): ConcatConv2d(
                    (_layer): Conv2d(4, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                  )
                  (1): ConcatConv2d(
                    (_layer): Conv2d(65, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                  )
                  (2): ConcatConv2d(
                    (_layer): Conv2d(65, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                  )
                  (3): ConcatConv2d(
                    (_layer): Conv2d(65, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                  )
                )
                (activation_fns): ModuleList(
                  (0): Softplus(beta=1, thres